In [1]:
test_senteces = ['today is not that bad', 'today is so bad', 'so good']
model_name = 'distilbert-base-uncased-finetuned-sst-2-english'
import os#环境代理设置
os.environ["http_proxy"] = "http://127.0.0.1:7890"
os.environ["https_proxy"] = "http://127.0.0.1:7890"

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

/home/jiangzirou/data/miniconda3/envs/PyTroch/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

In [4]:
input = tokenizer(test_senteces, truncation=True, padding=True,return_tensors='pt')

In [5]:
input

{'input_ids': tensor([[ 101, 2651, 2003, 2025, 2008, 2919,  102],
        [ 101, 2651, 2003, 2061, 2919,  102,    0],
        [ 101, 2061, 2204,  102,    0,    0,    0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 0, 0, 0]])}

In [6]:
tokenizer.convert_ids_to_tokens(input["input_ids"][0])

['[CLS]', 'today', 'is', 'not', 'that', 'bad', '[SEP]']

In [7]:
tokenizer.convert_tokens_to_ids([op for op in tokenizer.special_tokens_map.values()])

[100, 102, 0, 101, 103]

In [8]:
import torch
import torch.nn.functional as F

In [9]:
model.config

DistilBertConfig {
  "_attn_implementation_autoset": true,
  "_name_or_path": "distilbert-base-uncased-finetuned-sst-2-english",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "finetuning_task": "sst-2",
  "hidden_dim": 3072,
  "id2label": {
    "0": "NEGATIVE",
    "1": "POSITIVE"
  },
  "initializer_range": 0.02,
  "label2id": {
    "NEGATIVE": 0,
    "POSITIVE": 1
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.46.3",
  "vocab_size": 30522
}

In [10]:
def fc(**kwargs):
    for k,v in kwargs.items():
        print(k,v)
fc(**input)

input_ids tensor([[ 101, 2651, 2003, 2025, 2008, 2919,  102],
        [ 101, 2651, 2003, 2061, 2919,  102,    0],
        [ 101, 2061, 2204,  102,    0,    0,    0]])
attention_mask tensor([[1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 0, 0, 0]])


In [11]:
model.config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}

In [12]:
with torch.no_grad():
    outputs = model(**input)
    print(outputs)
    scores = F.softmax(outputs.logits,dim=-1)
    print(scores)
    labels = torch.argmax(scores,dim=-1)
    print(labels)
    labels = [model.config.id2label[id] for id in labels.tolist()]
    print(labels)

SequenceClassifierOutput(loss=None, logits=tensor([[-3.4620,  3.6118],
        [ 4.7508, -3.7899],
        [-4.1938,  4.5566]]), hidden_states=None, attentions=None)
tensor([[8.4632e-04, 9.9915e-01],
        [9.9980e-01, 1.9531e-04],
        [1.5837e-04, 9.9984e-01]])
tensor([1, 0, 1])
['POSITIVE', 'NEGATIVE', 'POSITIVE']


In [13]:
tokenizer.model_input_names

['input_ids', 'attention_mask']